# Terza Esercitazione
## Infiltrazione dell'acqua in una colonna di suolo

L'obiettivo della terza parte dell'esercitazione è quello di studiare l'infiltrazione dell'acqua all'interno di una colonna a più strati di suolo attraverso l'equazione di Richards.Quest'ultima rappresenta infatti l'equazione di conservazione della massa all'interno del suolo e tiene conto di un termine gravitativo, uno avvettivo ed uno diffusivo.

In caso, infatti, di disequilibrio nel carico idraulico, si viene a creare un moto all'interno del suolo che viene compensato dal movimento dell'acqua. La trattazione in questo caso sarà discreta: vengono quindi considerati i valori in alcuni punti nel dominio e dev'essere a questo fine introdotta una griglia. Per avere risultati più precisi sarà poi necessario infittire i punti che formano la griglia.
L'equazione di Richards è in generale valida per i suoli insaturi, può però essere anche estesa a quelli saturi
è necessario, per eseguire il programma, assegnare un dominio di calcolo, la griglia di calcolo e un dominio iniziale.

In [22]:
import os
import pandas as pd

from geoframepy.whetgeo1d import grid_creator
from geoframepy.whetgeo1d import grid_to_netcdf

import warnings
warnings.filterwarnings('ignore')


project_path = os.path.dirname(os.getcwd())


Per creare la griglia definiamo dei file .csv che vengono poi riportati come segue:

In [23]:
grid_input_file_name = project_path + "/data/Grid_input/ex00_grid.csv"

ic_input_file_name = project_path + "/data/Grid_input/ex00_ic.csv"

parameter_input_file_name = project_path + "/data/Grid_input/Richards_VG.csv"

dictionary_input_file_name = project_path + "/data/Grid_input/dictionary.csv"

grid_type = 'classical'

dz_min = -9999 # not required for grid_type = 'classical'

dz_max = -9999 # not required for grid_type = 'classical'

b = -9999      # not required for grid_type = 'classical'

psi_interp_model = "linear"

T_interp_model = "linear"



output_file_name =  project_path + "/data/Grid_NetCDF/ex00_grid.nc"

output_title = '''Computational grid for ex00.
                  '''
output_summary = '''

'''

output_date = ''

output_institution = 'GEOframe'


Per i parametri del suolo si utilizza la parametrizzazione di Van Genuchten. Per quello che riguarda invece la condizione iniziale si parte di solito assegnando una condizione idrostatica, di equlibrio, in modo che l'evoluzione poi nel tempo dipenda solo dalle condizioni al contorno

In [24]:
data_grid = pd.read_csv(grid_input_file_name)
data_ic = pd.read_csv(ic_input_file_name)
data_parameter = pd.read_csv(parameter_input_file_name, comment='#')
data_dictionary = pd.read_csv(dictionary_input_file_name)

[KMAX, eta, eta_dual, space_delta, z, z_dual, control_volume] = grid_creator.grid1D(data_grid, dz_min, b, dz_max, grid_type, shallow_water=False)

[psi_0, T_0] = grid_creator.set_initial_condition(data_ic, eta, psi_interp_model, T_interp_model, shallow_water=False)

control_volume_index = grid_creator.calibration_point_index(data_grid, eta)

[equation_state_ID, parameter_ID, theta_s, theta_r, par_1, par_2, par_3, par_4,
par_5, alpha_ss, beta_ss, ks] = grid_creator.set_parameters_richards(data_grid, data_parameter, data_dictionary, KMAX, eta)

grid_to_netcdf.write_grid_netCDF_richards(eta, eta_dual, z, z_dual, space_delta, control_volume, control_volume_index, psi_0, T_0, equation_state_ID, parameter_ID, KMAX,
                  theta_s, theta_r, par_1, par_2, par_3, par_4, par_5, alpha_ss, beta_ss, ks,
                  output_file_name, output_title, output_institution, output_summary, output_date, grid_input_file_name, parameter_input_file_name)



***SUCCESS writing!  C:\Users\titab\Desktop\Test_GEOframe_verona\OMS_Project_WHETGEO1D_Idrologia_ay_21_22/data/Grid_NetCDF/ex00_grid.nc


In [28]:
import static oms3.SimBuilder.instance as OMS3
import static oms3.SimBuilder.*
def home = oms_prj

#tutti i dati di input sono contenuti nelle prime righe del simfile

def richardsGridFileName = "$oms_prj//data/Grid_NetCDF/ex00_GSS2021.nc"

def typeClosureEquation = "{Van Genuchten}"

def typeEquationState = "{Van Genuchten}"

def typeUHCModel = "{Mualem Van Genuchten}"

def typeUHCTemperatureModel = "notemperature"

def interfaceHydraulicConductivityModel = "max"

def topBCType = "Top Dirichlet"

def bottomBCType = "Bottom free drainage"

def stationID = "0"

def startDate= "2020-01-01 00:00"

def endDate="2020-01-02 00:00"

def tTimestep = 1

def topBCFileName = "$oms_prj//data/Timeseries/ex00_water_ponding.csv"

def bottomBCFileName = "$oms_prj//data/Timeseries/ex00_free_drainage.csv"

def saveDatesFileName = "$oms_prj//data/Timeseries/ex00_save_dates.csv"

def outputFileName = "$oms_prj//output/ex00.nc"


def writeFrequency = 10000


solver = OMS3.sim(name: "Integrate PDE", {

resource "$oms_prj/dist"

	model(while : "reader_data_topBC.doProcess"  ){

		components {
			
			// reads of the grid.nc file
			"readNetCDF" "it.geoframe.blogpsot.netcdf.monodimensionalproblemtimedependent.ReadNetCDFRichardsGrid1D"
			
			// solve Richards' equation
			"solver" "it.geoframe.blogspot.whetgeo1d.richardssolver.RichardsSolver1DMain"
			
			// time-series reader
			"reader_data_topBC"         "org.hortonmachine.gears.io.timedependent.OmsTimeSeriesIteratorReader"
			"reader_data_bottomBC"      "org.hortonmachine.gears.io.timedependent.OmsTimeSeriesIteratorReader"
			"reader_data_save_dates"    "org.hortonmachine.gears.io.timedependent.OmsTimeSeriesIteratorReader"
			
			// this component stores solver's output during the simulation
			"buffer" "it.geoframe.blogspot.buffer.buffertowriter.RichardsBuffer1D"
			
			// output writer in a NetCDF format
			"writeNetCDF" "it.geoframe.blogpsot.netcdf.monodimensionalproblemtimedependent.WriteNetCDFRichards1DDouble"
		
		}
		
		parameter {

			"readNetCDF.richardsGridFilename" "${richardsGridFileName}"


			"solver.typeClosureEquation" "${typeClosureEquation}"
			

			"solver.typeEquationState" "${typeEquationState}"


			"solver.typeUHCModel" "${typeUHCModel}"


			"solver.typeUHCTemperatureModel" "${typeUHCTemperatureModel}"

            
			"solver.interfaceHydraulicConductivityModel" "${interfaceHydraulicConductivityModel}"
						

			"solver.topBCType"  "${topBCType}"

            
			"solver.bottomBCType"  "${bottomBCType}"

            
			"solver.stationID"  "${stationID}"
			

			"solver.newtonTolerance" "0.00000000001"


			"solver.nestedNewton" "1"
			
			"solver.picardIteration" "2"


			"solver.tTimeStep" tTimestep*60


			"solver.timeDelta" tTimestep*60

			"reader_data_topBC.file"             "${topBCFileName}"
			"reader_data_topBC.idfield"          "ID"  
			"reader_data_topBC.tStart"           "${startDate}"
			"reader_data_topBC.tEnd"             "${endDate}"
			"reader_data_topBC.tTimestep"        "${tTimestep}"
			"reader_data_topBC.fileNovalue"      "-9999"

			"reader_data_bottomBC.file"             "${bottomBCFileName}"
			"reader_data_bottomBC.idfield"          "ID"  
			"reader_data_bottomBC.tStart"           "${startDate}"
			"reader_data_bottomBC.tEnd"             "${endDate}"
			"reader_data_bottomBC.tTimestep"        "${tTimestep}"
			"reader_data_bottomBC.fileNovalue"      "-9999"
			
			"reader_data_save_dates.file"             "${saveDatesFileName}"
			"reader_data_save_dates.idfield"          "ID"  
			"reader_data_save_dates.tStart"           "${startDate}"
			"reader_data_save_dates.tEnd"             "${endDate}"
			"reader_data_save_dates.tTimestep"        "${tTimestep}"
			"reader_data_save_dates.fileNovalue"      "-9999"
			
			
			"buffer.writeFrequency"                   "${writeFrequency}"

			// folder path for the output files
			"writeNetCDF.fileName" "${outputFileName}"
			// here you can enter a string to describe your simulation
			"writeNetCDF.briefDescritpion"               "WHETGEO-1D ex00."
			"writeNetCDF.writeFrequency"                 "${writeFrequency}"
			"writeNetCDF.timeUnits"                      "Minutes since 01/01/1970 00:00:00 UTC"
			"writeNetCDF.timeZone"                       "UTC"
			"writeNetCDF.fileSizeMax"                    "10000"
			"writeNetCDF.outVariables"                   "{darcyVelocity}"
			"writeNetCDF.pathGrid"		                 "${richardsGridFileName}"
			"writeNetCDF.pathTopBC"		                 "${topBCFileName}"
			"writeNetCDF.pathBottomBC"	                 "${bottomBCFileName}"
			"writeNetCDF.topBC"		                     "${topBCType}"
			"writeNetCDF.bottomBC"	                     "${bottomBCType}"
			"writeNetCDF.swrcModel"                      "${typeClosureEquation}"
			"writeNetCDF.soilHydraulicConductivityModel" "${typeUHCModel}"
			"writeNetCDF.interfaceConductivityModel"     "${interfaceHydraulicConductivityModel}"
			
		}

		connect{


			"readNetCDF.z"                      "solver.z"
			"readNetCDF.spaceDelta"             "solver.spaceDeltaZ"
			"readNetCDF.psiIC"                  "solver.psiIC"
			"readNetCDF.temperature"            "solver.temperature"
			"readNetCDF.controlVolume"          "solver.controlVolume"
			"readNetCDF.Ks"                     "solver.ks"
			"readNetCDF.thetaS"                 "solver.thetaS"
			"readNetCDF.thetaR"                 "solver.thetaR"
			"readNetCDF.par1SWRC"               "solver.par1SWRC"
			"readNetCDF.par2SWRC"               "solver.par2SWRC"
			"readNetCDF.par3SWRC"               "solver.par3SWRC"
			"readNetCDF.par4SWRC"               "solver.par4SWRC"
			"readNetCDF.par5SWRC"               "solver.par5SWRC"
			"readNetCDF.alphaSS"                "solver.alphaSpecificStorage"
			"readNetCDF.betaSS"                 "solver.betaSpecificStorage"
			"readNetCDF.equationStateID"        "solver.inEquationStateID"
			"readNetCDF.parameterID"            "solver.inParameterID"

			"reader_data_topBC.tCurrent"        "solver.inCurrentDate"
			"reader_data_topBC.outData"         "solver.inTopBC"
			"reader_data_bottomBC.outData"      "solver.inBottomBC"
			"reader_data_save_dates.outData"    "solver.inSaveDate"
			
			
			
			// input buffer
			"solver.inCurrentDate"        "buffer.inputDate"
			"solver.doProcessBuffer"      "buffer.doProcessBuffer"
			"solver.outputToBuffer"       "buffer.inputVariable"

			
			// input writeNetCDF
			"reader_data_topBC.doProcess"        "writeNetCDF.doProcess"
			
			"buffer.myVariable"                  "writeNetCDF.variables"
			
			"readNetCDF.eta"                     "writeNetCDF.spatialCoordinate"
			"readNetCDF.etaDual"                 "writeNetCDF.dualSpatialCoordinate"
			"readNetCDF.controlVolume"           "writeNetCDF.controlVolume"
			"readNetCDF.psiIC"                   "writeNetCDF.psiIC"
			"readNetCDF.temperature"             "writeNetCDF.temperature"
			
			
		}
	}

	}

)


SyntaxError: invalid syntax (744641040.py, line 1)

In [26]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib.style as style 
import matplotlib.colors as cl
import matplotlib.dates as mdates
import matplotlib
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

style.available
style.use('seaborn-whitegrid')

nice_fonts = {
        "legend.frameon": True,
        "legend.fancybox": True,
        "legend.facecolor": "white",
        "axes.edgecolor": "0.8",
        "axes.linewidth": 0.6,
        "grid.linewidth":0.4,
        "text.usetex": True,
        "font.family": "serif",
        "axes.labelsize": 16,
        "font.size": 16,
        "legend.fontsize": 14,
        "xtick.labelsize": 14,
        "ytick.labelsize": 14,
}

my_cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["#ece7f2","#d0d1e6","#a6bddb","#74a9cf","#3690c0","#0570b0","#045a8d","#023858"])
colors = ['#a6cee3', '#1f78b4', '#b2df8a', '#33a02c', '#fb9a99', '#e31a1c', '#fdbf6f', '#ff7f00']

plt.rcParams.update(nice_fonts)

In [18]:
ds = xr.open_dataset(project_path + '/output/ex00_0000.nc')

Per vedere se la simulazione è andata a buon fine si osserva l'errore della macchina che sarà sempre, per quanto molto piccolo, diverso da zero:

In [19]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10/1.618)) 
ds.error.plot(ax=ax)
plt.show()

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 720x444.994 with 1 Axes>